# Model training

## Load Python packages

In [1]:
import numpy as np
from scipy import stats
import session_info
import pdb
from sklearn.metrics import r2_score
import shutil
import pandas as pd
import re
import os
from Bio.Seq import Seq
import gc
import matplotlib.pyplot as plt
import datetime
import socket
import time
import json
from collections import OrderedDict
import random as python_random

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from torch.nn.parameter import Parameter
import math
import pickle as pk
import sys

## Parameters

In [3]:
RUNTIME = 'none'
ARGS = {
  'model_id' : 'm20220727e',
  'global_seed' : 123,
  'shuffle_size' : 1000,
  'max_width' : 100,
  'head_len' : 17,
  'tail_len' : 13,
  'pct_ds' : 1, # % of total data for training/testing,
  'train_split' : 0.95,
  'alphabets' : {'A' : 0, 'C' : 1, 'G' : 2, 'T' : 3, 'N' : 4, 'M' : 5},
  'initial_lr' : 1e-15,
  'max_lr' : 23e-5,
  'initial_epoch': 0,
  'epochs' : 20,
  'batch_size' : 512,
  'dropout_rate' : 0.1,
  'kmer': 10,
  'strides' : 1,
  'embedding_dim' : 512,
  'num_heads' : 8,
  'ff_mult' : 4,
  'num_projectors' : 32,
  'n_blocks_regressor' : 4,
  'warmup_steps' : 12500, # ~ 1 epoch
  'mask_ratio' : 0.05,
  'remote_sample_submission_file' : 'https://raw.githubusercontent.com/de-Boer-Lab/DREAM-2022/main/sample_submission.json',
  'eval' : False,
  'device':'cuda:1',
  'local_data_dir' : '/Data1/PGE/torch_ti/data/'
}

In [4]:
with open(ARGS['local_data_dir']+"data.pk","rb") as fr:
    data = pk.load(fr)

### Set seeds

In [5]:
np.random.seed(ARGS['global_seed'])
torch.manual_seed(ARGS['global_seed'])
python_random.seed(ARGS['global_seed'])

### pearson_r

In [6]:
def pearson_r(x, y):
    x = torch.tensor(x,dtype=torch.float32)
    y = torch.tensor(y,dtype=torch.float32)
    mx = torch.mean(x, axis = 0, keepdims = True)
    my = torch.mean(y, axis = 0, keepdims = True)
    xm = x - mx
    ym = y - my
    t1_norm = F.normalize(xm, p=2, dim=0)
    t2_norm = F.normalize(ym, p=2, dim=0)
    return torch.sum(torch.mul(t1_norm, t2_norm))

In [7]:
x = np.random.rand(100)
y = np.random.rand(100)
print('pearson r (stats.pearsonr): {}'.format(stats.pearsonr(x, y)[0]))
print('pearson r (pearson_r): {}'.format(pearson_r(torch.unsqueeze(torch.Tensor(x),1), torch.unsqueeze(torch.Tensor(y),1))))

pearson r (stats.pearsonr): -0.09270195576139685
pearson r (pearson_r): -0.09270194172859192


/tmp/ipykernel_4574/2976460682.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x,dtype=torch.float32)
/tmp/ipykernel_4574/2976460682.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y,dtype=torch.float32)


## Read data

In [8]:
n = int(len(data['seq']))
n_train = int(n * ARGS['train_split'])
print('downsampled dataset size: %d' % (n))
print('training dataset size: %d' % (n_train))

downsampled dataset size: 6737568
training dataset size: 6400689


In [9]:
train_data = {'seq':data['seq'][:n_train],'expression':data['expression'][:n_train]}
val_data = {'seq':data['seq'][n_train:],'expression':data['expression'][n_train:]}

print('# training samples: %d' % (len(train_data['seq'])))
print('# val samples: %d' % (len(val_data['seq'])))

# training samples: 6400689
# val samples: 336879


# DataLoader & TestSet Preprocessing

In [10]:

class train_loader(object):
    def __init__(self, data):
        self.kmer = 10
        self.strides = 1
        self.input_dim = 6
        self.seq = data['seq']
        self.data = torch.tensor(data['seq'])
        self.data_label = data['expression']

        x = F.one_hot(self.data.to(torch.int64), self.input_dim)   # output = (b,seq,embed)
        self.data2 = x.transpose(1,2).numpy()

#        print('trn_data_shape : ',self.data2.shape)
    def __getitem__(self, index):

        return torch.FloatTensor(self.data2[index]), self.data_label[index], torch.FloatTensor(self.seq[index]) 
    
    def __len__(self):
        return len(self.data)

class test_loader(object):
    def __init__(self,args):
        lines = open("/Data1/PGE/torch_ti/filtered_test_data_with_MAUDE_expression.txt", "r").read().splitlines()
        data = [x.split('\t')[0] for x in lines]
        data_label = [x.split('\t')[1] for x in lines]
        df = pd.DataFrame()
        df['dna'] = data
        df['expression'] = data_label
        df['dna'] = df['dna'].astype('string')
        df['len'] = df['dna'].str.len()
        print('number of unique sequences in the first {} positions: {}'.format(args['head_len'], len(df['dna'].str[:args['head_len']].unique())))
        print('number of unique sequences in the last {} positions: {}'.format(args['tail_len'], len(df['dna'].str[-args['tail_len']:].unique())))
        df['dna'] = df['dna'].str[args['head_len']:]
        df['dna'] = df['dna'].str[:-args['tail_len']]
        df['len'] = df['dna'].str.len()
        assert all(df['len'] <= args['max_width'])
        
        df['dna'] = df['dna'].str.pad(width = args['max_width'], side = 'both', fillchar = 'N')
        df['dna'] = df['dna'] + df['dna'].apply(lambda x: str(Seq(x).reverse_complement())).astype('string')
        
        input_dim = int(6) # A,C,G,T,N,M
        n_positions = int(args['max_width'] * 2)
        self.dna = np.empty((0, n_positions), np.uint8)
        for x in np.array_split(df['dna'], 10): # split data into chunks
            y = np.array(x.apply(list))
            y = np.vstack(y)
            y = np.vectorize(ARGS['alphabets'].get)(y)
            y = y.astype(np.uint8)
            print(y.shape)
            self.dna = np.append(self.dna, y, axis = 0)
        print(self.dna.shape)
        self.expression = df['expression'].astype('float32').to_numpy()
        expression_std = np.std(self.expression)
        expression_mean = np.mean(self.expression)
        self.expression = (self.expression - expression_mean) / expression_std
        
        print(self.expression.shape)
        
        
        self.kmer = 10
        self.strides = 1
        self.input_dim = 6
        self.data = torch.tensor(self.dna)
        
        x = F.one_hot(self.data.to(torch.int64), self.input_dim)   # output = (b,seq,embed)
        self.data2 = x.transpose(1,2).numpy()  # output = (b,embed,seq)
        
#        print('test_data_shape : ',self.data2.shape)
        
        
    def __getitem__(self, index):

        return torch.FloatTensor(self.data2[index]), self.expression[index], torch.FloatTensor(self.dna[index]) 
    
    def __len__(self):
        return len(self.dna)

In [11]:
from tqdm import tqdm
from prixfixe.unlockdna import (UnlockDNA_CoreBlock,
                      UnlockDNA_FirstLayersBlock,
                      UnlockDNA_FinalLayersBlock)

from prixfixe.prixfixe import PrixFixeNet


class RegressorModel(nn.Module):
	def __init__(self, args,**kwargs):
		super(RegressorModel, self).__init__()
		## regressor
		self.arg = args
		first = UnlockDNA_FirstLayersBlock(in_channels=6,
                                   out_channels=512, 
                                   seqsize=200)
		core = UnlockDNA_CoreBlock(in_channels=512,
                         out_channels =512,
                         seqsize=232)        
		final = UnlockDNA_FinalLayersBlock(in_channels=512, 
                                 seqsize=232)       
		        
		self.regressor = PrixFixeNet(first=first,core=core,final=final).cuda()
		self.mse_loss = nn.MSELoss(reduction='none').cuda()
		self.scc_loss = nn.CrossEntropyLoss( reduction='none').cuda()
		self.optim           = torch.optim.Adam(self.regressor.parameters(), lr = args['initial_lr'], betas=(0.9, 0.98), eps=1e-08)
		self.scheduler       = torch.optim.lr_scheduler.OneCycleLR(self.optim, max_lr=ARGS['max_lr'],pct_start = 0.04, 
                                                                   steps_per_epoch=ARGS['warmup_steps'], epochs=ARGS['epochs']+5,anneal_strategy='cos')
		print(time.strftime("%m-%d %H:%M:%S") + " Model para number(백만) = %.2f"%(sum(param.numel() for param in self.regressor.parameters()) / 1024 / 1024))

	def train_network(self, epoch, loader):
		self.train()
		## Update the learning rate based on the current epoch
		if epoch > 0 :
			self.scheduler.step((epoch - 1)*int(n_train/self.arg['batch_size']))
			print('LR : ',self.scheduler.get_last_lr()[0])
		index, loss = 0, 0
		for num, (data, labels, sequence) in tqdm(enumerate(loader, start = 1)):

			self.zero_grad()
			seq, mask = self.regressor.masking(sequence)
			labels = labels.cuda()
			expression, seq_pred = self.regressor.forward(data.cuda()) 
			loss_expression = self.mse_loss(labels.to(torch.float32), expression.squeeze(1).to(torch.float32))
			loss_seq = mask.cuda() * self.scc_loss(seq_pred,sequence.long().cuda()) #.long()
			loss_seq = torch.sum(loss_seq) / (torch.sum(mask.cuda()) + 1)
			nloss = (loss_expression.to(torch.float32) + loss_seq.to(torch.float32)).mean().to(torch.float32)
			
			nloss.backward()
			self.optim.step()
			self.scheduler.step()
			lr = self.scheduler.get_last_lr()[0]
			index += len(labels)
			loss += nloss.detach().cpu().numpy()
			if num % 500 == 0 :
				sys.stderr.write(time.strftime("%m-%d %H:%M:%S") + \
				" [%2d] Lr: %5f, Training: %.2f%%, "    %(epoch, lr, 100 * (num / loader.__len__())) + \
				" Loss: %.5f \r"        %(loss/(num)))
				sys.stderr.flush()
				sys.stdout.write("\n")

		return loss/num, lr 

	def eval_network(self, loader):
		self.eval()
		exp = []
		real_exp = []
		for idx, (data,labels,_) in tqdm(enumerate(loader)):
			data_1 = torch.FloatTensor(data).cuda()
			with torch.no_grad():
				expression, _ = self.regressor.forward(data_1)
			if len(expression.shape) > 1 :
				expression = expression.reshape(-1)
				expression = expression.detach().cpu().numpy()
				labels = labels.detach().cpu().numpy() 
			exp.append(expression)
			real_exp.append(labels)
			
		# Coumpute Metric
		exp = np.array(exp).reshape(-1)
		real_exp = np.array(real_exp).reshape(-1)
		PR = pearson_r(exp, real_exp)

		return PR

	def save_parameters(self, path):
		torch.save(self.state_dict(), path)

	def load_parameters(self, path):
		self_state = self.state_dict()
		loaded_state = torch.load(path)
		for name, param in loaded_state.items():
			origname = name
			if name not in self_state:
				name = name.replace("module.", "")
				if name not in self_state:
					print("%s is not in the model."%origname)
					continue
			if self_state[name].size() != loaded_state[origname].size():
				print("Wrong parameter length: %s, model: %s, loaded: %s"%(origname, self_state[name].size(), loaded_state[origname].size()))
				continue
			self_state[name].copy_(param)

## Model fitting

### Epoch 1 

initial epoch: 1

In [ ]:
torch.cuda.empty_cache()
import gc
gc.collect()
import argparse, glob, os, torch, warnings, time


model_save_path = "exps/model_reproducing_3module"
score_save_path = "exps/score_reproducing_3module.txt"
os.makedirs(model_save_path,exist_ok = True)

device = ARGS['device']
torch.cuda.set_device(device)
print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())


## Define the data loader
trainloader = train_loader(train_data)
trainLoader = torch.utils.data.DataLoader(trainloader, batch_size = ARGS['batch_size'], shuffle = True, num_workers = 0, drop_last = True)
valloader = train_loader(val_data)
valLoader = torch.utils.data.DataLoader(valloader, batch_size = ARGS['batch_size'], shuffle = True, num_workers = 0, drop_last = True)
testloader = test_loader(ARGS)
testLoader = torch.utils.data.DataLoader(testloader, batch_size = ARGS['batch_size'], shuffle = True, num_workers = 0, drop_last = True)

## Search for the exist models
modelfiles = glob.glob('%s/model_0*.model'%model_save_path)
modelfiles.sort()

## Otherwise, system will try to start from the saved model&epoch
if len(modelfiles) >= 1:
	print("Model %s loaded from previous state!"%modelfiles[-1])
	epoch = int(os.path.splitext(os.path.basename(modelfiles[-1]))[0][6:]) + 1
	s = RegressorModel(ARGS)
	s.load_parameters(modelfiles[-1])
	eval_pr_ = s.eval_network(testLoader)
	print('Previous Eval Pearson_R : ',eval_pr_)
## Otherwise, system will train from scratch
else:
	epoch = 1
	s = RegressorModel(ARGS)

pr = []
eval_pr = []
score_file = open(score_save_path, "a+")

while(1):
	## Training for one epoch
	loss, lr = s.train_network(epoch = epoch, loader = trainLoader)

	## Evaluation every [test_step] epochs
	if epoch % 1 == 0:
		s.save_parameters(model_save_path + "/model_%04d.model"%epoch)
		pr.append(s.eval_network(valLoader))
		print(time.strftime("%Y-%m-%d %H:%M:%S"), "%d epoch, Pearson_R %2.2f%%, bestPearson_R %2.2f%%"%(epoch, pr[-1], max(pr)))
		score_file.write("%d epoch, LR %f, LOSS %f, Pearson_R %2.2f%%, bestPearson_R %2.2f%%\n"%(epoch, lr, loss, pr[-1], max(pr)))
		score_file.flush()
		if pr[-1] == max(pr) :
			s.save_parameters(model_save_path + "/model_best.model")
			eval_pr.append(s.eval_network(testLoader))
			print(time.strftime("%Y-%m-%d %H:%M:%S"), "%d epoch, Eval_Pearson_R %2.2f%%, Best_Eval_Pearson_R %2.2f%%"%(epoch, eval_pr[-1], max(eval_pr)))
			score_file.write("%d epoch, LR %f, LOSS %f, Pearson_R %2.2f%%, Best_Eval_Pearson_R %2.2f%%\n"%(epoch, lr, loss, eval_pr[-1], max(eval_pr)))
			score_file.flush()

	if epoch >= ARGS['epochs']:
		quit()

	epoch += 1


Device: cuda:1
Current cuda device: 1
number of unique sequences in the first 17 positions: 1
number of unique sequences in the last 13 positions: 1
(7111, 200)
(7111, 200)
(7111, 200)
(7110, 200)
(7110, 200)
(7110, 200)
(7110, 200)
(7110, 200)
(7110, 200)
(7110, 200)
(71103, 200)
(71103,)


/opt/conda/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
/opt/conda/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.

03-28 07:44:22 Model para number(백만) = 17.19
LR :  9.200000000000007e-06


500it [15:45,  1.90s/it]03-28 08:00:08 [ 1] Lr: 0.000010, Training: 4.00%,  Loss: 0.97563 

1000it [31:32,  1.89s/it]3-28 08:15:55 [ 1] Lr: 0.000013, Training: 8.00%,  Loss: 0.91836 

...

4500it [2:22:17,  1.90s/it]04-02 11:46:01 [19] Lr: 0.000041, Training: 36.00%,  Loss: 0.29599 

5000it [2:38:05,  1.89s/it]04-02 12:01:49 [19] Lr: 0.000040, Training: 40.00%,  Loss: 0.29623 

5500it [2:53:54,  1.89s/it]04-02 12:17:38 [19] Lr: 0.000040, Training: 44.00%,  Loss: 0.29676 

6000it [3:09:42,  1.96s/it]04-02 12:33:27 [19] Lr: 0.000039, Training: 48.00%,  Loss: 0.29680 

6280it [3:18:33,  1.89s/it]

## Model Evaluation

In [16]:
ARGS['eval'] = True

In [17]:
torch.cuda.empty_cache()
import gc
gc.collect()
import argparse, glob, os, torch, warnings, time


model_save_path = "exps/model_reproducing_3module"
score_save_path = "exps/score_reproducing_3module.txt"
os.makedirs(model_save_path,exist_ok = True)

device = ARGS['device']
torch.cuda.set_device(device)
print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())

testloader = test_loader(ARGS)
testLoader = torch.utils.data.DataLoader(testloader, batch_size = ARGS['batch_size'], shuffle = True, num_workers = 0, drop_last = True)

## Search for the exist models
modelfiles = glob.glob('%s/*.model'%model_save_path)
modelfiles.sort()
for i in modelfiles : 
    if 'best' in i :
        evalfile_path = i
print(evalfile_path)
## Only do evaluation, the initial_model is necessary
if ARGS['eval'] == True :
    s = RegressorModel(ARGS)
    print("Model %s loaded from previous state!"%evalfile_path)
    s.load_parameters(evalfile_path)
    PR = s.eval_network(testLoader)
    print("Evaluation Pearson R : %2.2f%%"%(PR))
    quit()


Device: cuda:1
Current cuda device: 1
number of unique sequences in the first 17 positions: 1
number of unique sequences in the last 13 positions: 1
(7111, 200)
(7111, 200)
(7111, 200)
(7110, 200)
(7110, 200)
(7110, 200)
(7110, 200)
(7110, 200)
(7110, 200)
(7110, 200)
(71103, 200)
(71103,)
exps/model_reproducing_3module/model_best.model
04-04 10:24:13 Model para number(백만) = 17.19
Model exps/model_reproducing_3module/model_best.model loaded from previous state!


138it [01:21,  1.69it/s]

Evaluation Pearson R : 0.95%
